In [1]:
import hsa
import numpy as np
import xarray as xr
import xarray.ufuncs as xu
from dask.diagnostics import ProgressBar
import scipy.stats as ss
import paths as ps
from datetime import datetime, timedelta
import bottleneck
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
lons = np.arange(180,310.1,0.5)
lats = np.arange(20,80.1,0.5)
nfa = hsa.NewForecastArray('mean','slp',24)
gefs = nfa.load_forecast(subset_lat=lats,subset_lon=lons)


In [3]:
variable = 'slp'
with open(ps.log_directory + 'current_run.txt', "r") as f:
    model_date=datetime.strptime(f.readlines()[-1][5:13],'%Y%m%d')
nfa = hsa.NewForecastArray('mean','slp',24)
gefs = nfa.load_forecast(subset_lat=lats,subset_lon=lons)    
mc = hsa.MClimate(model_date, variable, 24, percentage=10)

In [4]:
def xarr_interpolate(original, new, on='latlon'):
    if on == 'latlon':
        new_lat = [i for i in new.coords if 'lat' in i][0]
        new_lon = [i for i in new.coords if 'lon' in i][0]
        old_lat = [i for i in original.coords if 'lat' in i][0]
        old_lon = [i for i in original.coords if 'lon' in i][0]
        original_i = original.interp({old_lat : new[new_lat].values}).interp({old_lon : new[new_lon].values})
        return original_i
    else:
        raise Exception('latlon interpolation only works as of now...')

In [5]:
def hsa_n(variable):
    start = datetime.now()
    with open(ps.log_directory + 'current_run.txt', "r") as f:
        model_date=datetime.strptime(f.readlines()[-1][5:13],'%Y%m%d')
    for f in range(0,7,6):
        start = datetime.now()
        nfa = hsa.NewForecastArray('mean','slp',f)
        gefs = nfa.load_forecast(subset_lat=lats,subset_lon=lons)
        print(f'gefs took {(datetime.now()-start).total_seconds():.2f}')
        start = datetime.now()
        mc = hsa.MClimate(model_date, variable, f, percentage=10)
        mc_mu = xarr_interpolate(mc.generate(type='mean',dask=True),gefs)
        print(f'mu interpolate took {(datetime.now()-start).total_seconds():.2f}')
        start = datetime.now()
        mc_std = xarr_interpolate(mc.generate(type='sprd',dask=True),gefs)
        print(f'sigma interpolate took {(datetime.now()-start).total_seconds():.2f}')
    return mc_mu, gefs

In [6]:
lons = np.arange(180,310.1,0.5)
lats = np.arange(20,80.1,0.5)
mc_mu,gefs=hsa_n('slp')

gefs took 2.91
mu interpolate took 0.27
sigma interpolate took 0.25
gefs took 4.57
mu interpolate took 0.33
sigma interpolate took 0.24


In [7]:
gefs = gefs.drop(['step','meanSea','valid_time'])
new_stacked=xr.concat([mc_mu[[n for n in mc_mu][0]],gefs[[n for n in gefs][0]]],'time')
new_stacked = new_stacked.compute()
percentile = new_stacked.rank('time')/len(new_stacked['time'])

In [8]:
new_perc = percentile.where(np.logical_and(percentile >= percentile.isel(time=-1)-0.05,percentile <= percentile.isel(time=-1)+0.05),drop=True)

In [9]:
mc_std = xarr_interpolate(mc.generate(type='sprd',dask=True),gefs)

NameError: name 'mc' is not defined

In [ ]:
mc_std_da=mc_std[[n for n in mc_std][0]]

In [ ]:
dropped=mc_std_da.where(~np.isnan(new_perc[:-1]),drop=True)

In [ ]:
dropped.isel(lat=0,lon=0).plot.hist()

In [ ]:
nfa_sprd = hsa.NewForecastArray('sprd', 'slp', 24)
gefs_sprd = nfa_sprd.load_forecast(subset_lat=lats,subset_lon=lons)
gefs_sprd = gefs_sprd.where(np.logical_and(gefs_sprd.lon>=np.min(lons), gefs_sprd.lon<=np.max(lons)), drop=True)
gefs_sprd = gefs_sprd.where(np.logical_and(gefs_sprd.lat>=np.min(lats), gefs_sprd.lat<=np.max(lats)), drop=True)
 

In [ ]:
import cartopy.crs as ccrs
plt.figure(figsize=(15,10));
ax = plt.axes(projection=ccrs.Miller());

gefs_sprd['prmsl'].plot.contourf(ax=ax,transform=ccrs.PlateCarree())
ax.set_extent([-180,-50,20,70])
ax.coastlines();
plt.show()
plt.close('all')
percentile.isel(time=-1).plot.hist()

In [ ]:
hsa_test = (gefs_sprd.rename({'prmsl':'Pressure'})-dropped.mean('time'))/dropped.std('time')

In [ ]:
hsa_test

In [ ]:
plt.figure(figsize=(15,10));

ax = plt.axes(projection=ccrs.Miller());

hsa_test['Pressure'].plot.contourf(ax=ax,transform=ccrs.PlateCarree())
ax.set_extent([-180,-50,20,70])
ax.coastlines();
plt.show()
plt.close('all')

In [ ]:
hsa_test2=(0.99-(-0.99))*(hsa_test-hsa_test.min())/(hsa_test.max()-hsa_test.min()) + -0.99
hsa_test2 = np.arctanh(hsa_test2)

In [ ]:
import matplotlib.font_manager as fm

import cartopy.feature as cfeature

fpath = '/E1/taylorm/espr/misc/comfortaa/Comfortaa-Regular.ttf'
prop = fm.FontProperties(fname=fpath)
fpath_bold = '/E1/taylorm/espr/misc/comfortaa/Comfortaa-Bold.ttf'
prop_bold = fm.FontProperties(fname=fpath_bold)
fig, ax = plt.subplots(figsize=(20,10), subplot_kw={'projection': ccrs.Miller()})

levels = np.linspace(-3,3,13)

contour = gefs[[n for n in gefs][0]].plot.contour(
    ax=ax,transform=ccrs.PlateCarree(),
    add_colorbar=False,
    colors='k',
    alpha=0.8)
ax.clabel(contour, fmt='%4.0f', inline=1, fontsize=10)

hsa_test2.where(np.abs(hsa_test2.Pressure)>0.5)['Pressure'].plot.contourf(
    ax=ax,transform=ccrs.PlateCarree(), 
    levels=levels,
    add_colorbar=False,
    alpha=0.8)
date_title = hsa_test2.Pressure.valid_time.dt.strftime("%Y/%m/%d %Hz").values
step = hsa_test2.Pressure.step.values.astype("timedelta64[h]")/np.timedelta64(1, "h")
ax.add_feature(cfeature.NaturalEarthFeature(
    'cultural', 'admin_1_states_provinces_lines', '50m',
    edgecolor='gray', facecolor='none'))
ax.add_feature(cfeature.NaturalEarthFeature(
    'cultural', 'admin_1_states_provinces_lines', '50m',
    edgecolor='gray', facecolor='none'))
ax.set_title(f'HISTORICAL SPREAD ANOMALY',fontproperties=prop,fontsize=12,loc='left')
ax.set_title(f'FHOUR: {step:2.0f}',fontproperties = prop_bold,fontsize=10,loc='center')
ax.set_title(f'VALID: {date_title}', fontproperties=prop_bold,fontsize=10,loc='right')
ax.add_feature(cfeature.LAKES, facecolor='gray')
ax.add_feature(cfeature.BORDERS, edgecolor='gray')
ax.add_feature(cfeature.RIVERS, edgecolor='gray')
ax.add_feature(cfeature.OCEAN, facecolor='gray')
ax.set_extent([-180,-50,20,65])
ax.coastlines(resolution='50m');
plt.show()
plt.close('all')


In [13]:
variable = 'wnd'
with open(ps.log_directory + 'current_run.txt', "r") as f:
    model_date=datetime.strptime(f.readlines()[-1][5:13],'%Y%m%d')
nfa_pwat = hsa.NewForecastArray('mean',variable,24)
gefs_pwat = nfa_pwat.load_forecast(subset_lat=lats,subset_lon=lons)    



IndexError: list index out of range

In [22]:
mc_mu

<xarray.Dataset>
Dimensions:             (lat: 121, lon: 261, time: 630)
Coordinates:
  * time                (time) datetime64[ns] 2012-10-04 ... 2012-10-24
  * lat                 (lat) float64 80.0 79.5 79.0 78.5 ... 21.0 20.5 20.0
  * lon                 (lon) float64 180.0 180.5 181.0 ... 309.0 309.5 310.0
Data variables:
    Precipitable_water  (time, lat, lon) float32 dask.array<chunksize=(1, 121, 261), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.0
    title:        Subset of data from 2nd-generation multi-decadal ensemble r...
    institution:  NOAA Earth System Research Laboratory (ESRL)
    source:       NCEP GFS v 9.01, T254L42.  Control initial conditions from ...
    references:   http://www.esrl.noaa.gov/psd/forecasts/reforecast2/index.html
    history:      Subset created 2019-10-03 05:52:08 UTC
    comment:      Original dataset generated on DOE's supercomputers at Lawre...

In [12]:
gefs_pwat

<xarray.Dataset>
Dimensions:     (lat: 121, lon: 261)
Coordinates:
    time        datetime64[ns] 2019-10-14T18:00:00
    step        timedelta64[ns] 1 days
    level       int64 0
  * lat         (lat) float64 80.0 79.5 79.0 78.5 78.0 ... 21.5 21.0 20.5 20.0
  * lon         (lon) float64 180.0 180.5 181.0 181.5 ... 309.0 309.5 310.0
    valid_time  datetime64[ns] 2019-10-15T18:00:00
Data variables:
    pwat        (lat, lon) float32 7.1 7.1 7.1 7.1 7.1 ... 43.4 41.4 39.4 37.9
Attributes:
    GRIB_edition:            2
    GRIB_centre:             kwbc
    GRIB_centreDescription:  US National Weather Service - NCEP 
    GRIB_subCentre:          2
    Conventions:             CF-1.7
    institution:             US National Weather Service - NCEP 

In [32]:
gefs_pwat = nfa.load_forecast(subset_lat=lats,subset_lon=lons)    

In [14]:
mc_wnd = hsa.MClimate(model_date, variable, 24, percentage=10)
mc_mu = xarr_interpolate(mc_wnd.generate(type='mean',dask=True),gefs)

FileNotFoundError: [Errno 2] No such file or directory: b'/home/taylorm/espr/reforecast/wnd_mean_son.nc'